In [11]:
import cv2
import mediapipe as mp
import numpy as np
import time
import dlib

In [12]:
import torch

import torch.nn as nn

import torch.nn.functional as F

import torchvision

from torchvision import datasets, transforms

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from model import model_static

In [13]:
facerec = dlib.face_recognition_model_v1('model_1113/dlib_face_recognition_resnet_model_v1.dat')

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

mp68=[143,116,123,147,192,210,211,208,199,428,431,430,416,376,352,345,372,
70,63,105,66,107,
336,296,334,293,300,
168,197,5,4,75,97,2,326,305,
33,160,158,133,153,144,362,385,387,263,373,
380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]

# Func

In [14]:
def find_faces(image,results):
    # Draw the face mesh annotations on the image.
    rects=[]
    shapes=dlib.full_object_detections()
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
# ------------------------------
            skipface=False
            points=dlib.points()
            p=face_landmarks.landmark
            ld68=[]
            for i in mp68:
                px=int(image.shape[1]*p[i].x)
                py=int(image.shape[0]*p[i].y)
                if px<0 or px>image.shape[1] or py<0 or py>image.shape[0]:
                    skipface=True
                    results.multi_face_landmarks.remove(face_landmarks)
                    break
                
                ld68.append([px,py])
                points.append(dlib.point(px,py))
            
            if skipface:
                continue

            cx=int((ld68[39][0]+ld68[42][0])/2)
            cy=ld68[29][1]


            x,y,w,h=cv2.boundingRect(np.array(ld68))
            if w<h:
                x-=int((h-w)/2)
                w=h
            elif w>h:
                y-=int((w-h)/2)
                h=w

            rcx=int(x+(w/2))
            rcy=int(y+(h/2))
            x+=(cx-rcx)
            y+=(cy-rcy)
# ------------------------------
    
            rect=((x,y),(x+w,y+h))
            rects.append(rect)
            shape=dlib.full_object_detection(dlib.rectangle(x,y,x+w,y+h),points)
            shapes.append(shape)
    return rects,shapes,[]

def rect_faces(image,results):
    # Draw the face mesh annotations on the image.
    rects=[]
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            
            p=face_landmarks.landmark
            ld68=[]
            for i in mp68:
                px=int(image.shape[1]*p[i].x)
                py=int(image.shape[0]*p[i].y)
                ld68.append([px,py])

            cx=int((ld68[39][0]+ld68[42][0])/2)
            cy=ld68[29][1]

            x,y,w,h=cv2.boundingRect(np.array(ld68))
            if w<h:
                x-=int((h-w)/2)
                w=h
            elif w>h:
                y-=int((w-h)/2)
                h=w

            rcx=int(x+(w/2))
            rcy=int(y+(h/2))
            x+=(cx-rcx)
            y+=(cy-rcy)
            
            rect=((x,y),(x+w,y+h))
            rects.append(rect)
    return rects


def encode_faces(img, shapes):
    face_descriptors = facerec.compute_face_descriptor(img, shapes)

    return np.array(face_descriptors)

In [15]:
def processFR(image,results,adminIndex):
    # 얼굴 인식 후 사용자 face index 업데이트
    # image - RGB
    
    rects, shapes, _ = find_faces(image,results)
    descs = encode_faces(image, shapes)
    admindiststemp={}

    
    for i, desc in enumerate(descs):
    
        found = False
        names=[]
        dists=[]
        for name, admindesc in adminfacedescs.items():
            dist = np.linalg.norm([desc] - admindesc, axis=1)
            if dist < 0.5:
                found = True
                names.append(name)
                dists.append(dist)

        if found:
            dists=np.array(dists)
            idx=np.argmin(dists)
            name=names[idx]
            
            if name in admindiststemp:
                
                if dists[idx]<admindiststemp[name][1]:
                    admindiststemp[name]=[i,dists[idx]]
            else:
                admindiststemp[name]=[i,dists[idx]]
            
    # 중복 검출 필터링
    for name, item in admindiststemp.items():
        adminIndex[item[0]]=name
        
    return rects

In [16]:
def processECD(image,rects,results,adminIndex):
    global lastecd
    global js
    global activatedAdmin
    global lastactivated
    
    face_imgs=None
    face_order=[]
    frame = Image.fromarray(image)
    
    if rects==-1:
        rects=rect_faces(image,results)
    if lastecd>=ecdterm:
        # 응시 인식 진행
        if activatedAdmin == '':
            # 모든 사용자 응시 인식
            for i,name in adminIndex.items():
                l=rects[i][0][0]
                t=rects[i][0][1]
                r=rects[i][1][0]
                b=rects[i][1][1]
                # expand a bit
                l -= (r-l)*0.2
                r += (r-l)*0.2
                t -= (b-t)*0.2
                b += (b-t)*0.2
                
                face = frame.crop(([l,t,r,b]))
                img = test_transforms(face)
                img.unsqueeze_(0)
                
                if face_imgs is None:
                    face_imgs=img
                else:
                    face_imgs = torch.cat([face_imgs, img])
                
                face_order.append(name)
                
        else:
            # 특정 사용자 응시 인식
            
            for i,name in adminIndex.items():
                if name==activatedAdmin:
                    l=rects[i][0][0]
                    t=rects[i][0][1]
                    r=rects[i][1][0]
                    b=rects[i][1][1]
                    # expand a bit
                    l -= (r-l)*0.2
                    r += (r-l)*0.2
                    t -= (b-t)*0.2
                    b += (b-t)*0.2
                    
                    face = frame.crop(([l,t,r,b]))
                    img = test_transforms(face)
                    img.unsqueeze_(0)
                    face_imgs=img
                    
                    face_order.append(name)
                    break
         
        if face_imgs is not None:
            # 모델 입력
            output = model(face_imgs.cuda())
            scores=torch.sigmoid(output)
            idx=torch.argmax(output)
            
            # 최대값 인덱스
            if activatedAdmin == '':
                # 사용자 활성화
                if scores[idx]>=0.85:
                    activatedAdmin=face_order[idx]
                    lastactivated=frametime
                    js=1

            else:
                if scores[idx]<0.85:
                    resetCheck()
                else:
                    lastactivated=frametime
        lastecd=frametime

In [17]:
def resetCheck():
    global lastactivated
    global activatedAdmin
    global js
    if frametime-lastactivated>=timeout:
        activatedAdmin=''
        js=0

# Init

In [18]:
# set up data transformation
test_transforms = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# load model weights
model_weight='data_1113/model_weights.pkl'
model = model_static(model_weight)
model_dict = model.state_dict()
snapshot = torch.load(model_weight)
model_dict.update(snapshot)
model.load_state_dict(model_dict)

model.cuda()
model.train(False)

loading saved model weights


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [19]:
adminList=['kim']

adminfacedescs = {}

activatedAdmin=''

adminsEcdHistory={}

ecdBuffer={}

# System State
ss=0
# 얼굴 인식 진행
# 모든 관리자 응시 인식 진행
# 응시 중인 사용자 여부

lastactivated=0

# Jesture State
js=0

# ECD state
es=0

# last ecd time
lastecd=0

ecdterm=0.3

timeout=2.0

# Previous People
pp=0
        

In [20]:
# 사용자 얼굴 등록
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.5) as face_mesh:

    img_bgr = cv2.imread('mingyu.jpg')
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)
    rects, shapes, cords = find_faces(img_rgb,results)
    adminfacedescs['kim']=encode_faces(img_rgb, shapes)[0]

# Demo

In [11]:
# Start Demo Program
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
pp=0
frametime=time.time()

with mp_face_mesh.FaceMesh(
    max_num_faces=5,
    refine_landmarks=True,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        start = time.time()
        
        success, image = cap.read()
        if not success:
            print("Empty camera frame.")
            break
            
        
        framegap=time.time()-frametime
        frametime=time.time()
        img_h, img_w, img_c = image.shape
        image.flags.writeable = False
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)
        
        if (results.multi_face_landmarks is None):
            pp=0
            adminIndex={}
            # 활성화 사용자 리셋 체크
            if activatedAdmin!='':
                resetCheck()
        
        elif (pp != len(results.multi_face_landmarks)):
            # 얼굴 인식 진행
            adminIndex={}
            rects=processFR(image,results,adminIndex)
            pp=len(results.multi_face_landmarks)
            
            # 응시 인식 진행
            processECD(image,rects,results,adminIndex)
        else:
            rects=-1
            # 응시 인식 진행
            processECD(image,rects,results,adminIndex)
            

        
        
        # Draw the face mesh annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        faceIdx=0
        if results.multi_face_landmarks:
              for face_landmarks in results.multi_face_landmarks:
                    mediapipe_lm=np.array([np.multiply([p.x,p.y],[img_w,img_h]).astype(int) for p in face_landmarks.landmark])
                    if faceIdx in adminIndex:
                        cv2.putText(image, f'{adminIndex[faceIdx]}', mediapipe_lm[152], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 20, 20), 2)
                        
                        
                    faceIdx+=1
        
        end = time.time()
        totalTime = end - start
        
        if activatedAdmin!='':
            cv2.putText(image, f'{activatedAdmin}, Welcome.', (0,20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 20, 20), 2)
            
        cv2.putText(image, f'{totalTime}', (0,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (25, 20, 20), 2)
        
        cv2.imshow('MediaPipe Face Mesh', image)
        wk=cv2.waitKey(1)
        if wk & 0xFF == 27: # esc
            break
            
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 